In [ ]:
# Looking at stock market data from companies with top ESG scores

In [ ]:
# Importing necessary packages

from pandas_datareader import data as web
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

plt.style.use('fivethirtyeight')

In [ ]:
# Setting parameters for data_reader package

stockStartDate = '2015-01-01'

today = datetime.today().strftime('%Y-%m-%d')

weights = np.array([0.25, 0.25, 0.25, 0.25])

In [ ]:
# Selecting assets to be used and loading into dataframe

assets=['APPL', 'GOOGL', 'MSFT']

df = pd.DataFrame()

for stock in assets:
    df[stock] = web.DataReader(stock, data_source='yahoo', start = stockStartDate, end = today)['Adj Close']

In [ ]:
# Visualizing the list of stocks in assets

title = 'Portfolio Adj. Close Price History'

my_stocks = df

for c in my_stocks.columns.values:
    plt.plot(my_stocks[c], label = c)

plt.title(title)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Adj. Price USD ($)', fontsize=18)
plt.legend(my_stocks.columns.values, loc='upper left')
plt.show()

**Correlation Analysis**

In [ ]:
# calculating returns
returns = df.pct_change()
print(returns)

# annualized covariance matrix
cov_matrix_annual = returns.cov() * 252
print(cov_matrix_annual)

In [ ]:
# calculate the portfolio variance
port_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
print(port_variance)

#calculate the portfolio volatility or standard deviation
port_volatility = np.sqrt(port_variance)
print(port_volatility)

In [ ]:
#calculate the annual portfolio returns
portfolioSimpleAnnualReturns = np.sum(returns.mean() * weights) * 252
print(portfolioSimpleAnnualReturns)

#show the expected annual returns, volatility (risk), variance
percent_var = str(round(port_variance, 2) * 100) + '%'
percent_vols = str(round(port_volatility, 2) * 100) + '%'
percent_ret = str(round(portfolioSimpleAnnualReturns, 2) * 100) + '%'
print('Expected Annual Return: ' + percent_ret)
print('Annual Volatility/Risk: ' + percent_vols)
print('Annual Variance: ' + percent_var)

**Portfolio Optimization**

In [ ]:
#Portfolio Optimization
#Calculate the expected returns and the annualized simple coviarance matrix of assets
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# optimize for max sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose = True)

In [ ]:
# Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = 15000)

allocation, leftover = da.lp_portfolio()
print('Discrete allocation:', allocation)
print('Funds remaining: ${:.2f}'.format(leftover))